In [6]:

import pandas as pd
fake = pd.read_csv('data/Fake.csv')
true = pd.read_csv('data/True.csv')
fake["category"]=1
true["category"]=0
df=pd.concat([fake,true]).reset_index(drop=True)
df= df.sample(frac=1).reset_index(drop=True)
df=df[["text","category"]]
blanks=[]
for index,text in df["text"].items():
    if text.isspace():
        blanks.append(index)
len(blanks)
df.drop(blanks,inplace=True)



In [3]:
import nltk

# List of required NLTK packages
packages = ['stopwords', 'wordnet', 'omw-1.4']

for pkg in packages:
    try:
        nltk.data.find(f'corpora/{pkg}')
    except LookupError:
        nltk.download(pkg)


[nltk_data] Downloading package wordnet to C:\Users\Ria
[nltk_data]     Pasricha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Ria
[nltk_data]     Pasricha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [35]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import re
nlp=spacy.load("en_core_web_sm")
lemma = WordNetLemmatizer()
list1=nlp.Defaults.stop_words
list2=stopwords.words('english')
Stopwords=set(set(list1)|set(list2))
print(len(Stopwords))


401


In [36]:
def clean_text(text):
    string=""
    text=text.lower()
    text=re.sub(r"i'm","i am",text)
    text=re.sub(r"he's","he is",text)
    text=re.sub(r"she's","she is",text)
    text=re.sub(r"that's","that is",text)
    text=re.sub(r"what's","what is",text)
    text=re.sub(r"where's","where is",text)
    text=re.sub(r"won't","will not",text)
    text=re.sub(r"can't","cannot",text)
    text=re.sub(r"\'ll'","will",text)
    text=re.sub(r"\'ve'","have",text)
    text=re.sub(r"\'re'","are",text)
    text=re.sub(r"\'d'","would",text)
    text=re.sub(r"[-()\"#!@$%^&*{}?.,:]"," ",text)
    text=re.sub(r"\s+"," ",text)
    text=re.sub('[^A-Za-z0-9]+',' ', text)
    
    for word in text.split():
     if word not in Stopwords:
         string+=lemma.lemmatize(word)+" "
    return string

In [37]:
df["text"]=df["text"].apply(clean_text)


In [38]:
from sklearn.model_selection import train_test_split


X=df["text"] 
y=df["category"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99, random_state=10)

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics

text_clf = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words='english',
        ngram_range=(1, 2),
        max_df=0.9,
        min_df=5,
        max_features=10000
    )),
    ("clf", LinearSVC(class_weight='balanced'))
])

text_clf.fit(X_train, y_train)

predictions = text_clf.predict(X_test)


In [40]:
from sklearn import metrics
print(metrics.classification_report(y_test,predictions))
print(metrics.accuracy_score(y_test,predictions))


              precision    recall  f1-score   support

           0       0.95      0.96      0.95     21215
           1       0.96      0.95      0.95     22610

    accuracy                           0.95     43825
   macro avg       0.95      0.95      0.95     43825
weighted avg       0.95      0.95      0.95     43825

0.9530176839703366


In [41]:
from newspaper import Article

def get_text_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"Error extracting article: {e}")
        return None
a=input(print('choose url or article'))
if(a=='article' or a=='Article'):
   s=input(print('enter article'))
   news_input = [s]
elif(a=='url' or a=='URL'):
    url=input(print('enter url'))
    article_text = get_text_from_url(url)
    news_input=[article_text]
else:
    print('error')


choose url or article
enter url


In [5]:
prediction = text_clf.predict(news_input)
if prediction== 0:
    print("This news is REAL.")
else:
    print("This news is FAKE.")

NameError: name 'text_clf' is not defined